Extract the Data from Amazon

In [1]:
import os
import pandas as pd
# Find the latest version of spark 3.2.x  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.2.3'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 261 kB in 1s (180 kB/s)
Reading package lis

In [2]:
# Install the PostgreSQL driver in our Colab environment
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-30 04:56:52--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.3’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.05MB/s    in 0.2s    

2022-11-30 04:56:53 (5.05 MB/s) - ‘postgresql-42.2.9.jar.3’ saved [914037/914037]



In [3]:
# Establish a Spark session and add the Postgres driver to the filepath
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read the first CSV file from an S3 bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
music_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Music_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)



# Show DataFrame
music_data_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|     384427924|Whatever's for Us...|           Music|          5|            0|          0|   N|                Y|          Five Stars|Love this CD alon...| 2015-08-31|
|         US|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|     831769051|Same Trailer Diff...|           Music|          5|    

In [5]:
# Drop null values
music_df = music_data_df.dropna(how='any')

#Drop duplicate values
music_df= music_df.drop_duplicates()

Count the number of rows

In [6]:
music_df.count()

4750852

In [8]:
# Create the review_id_table dataframe for the database
review_id_df = music_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1005BINTGKCVE|   18789304|B000002J6H|     480387571| 2013-01-29|
|R10067SN6FSII1|   32818060|B00005BCY9|     926227231| 2003-01-19|
|R100NGDZ6CAL8E|   32318800|B0000AB13N|     711057433| 2003-08-04|
|R100OLMAO039IL|   50673975|B000002SZA|     632006013| 2009-09-10|
| R100OMDUXE5VZ|   52503259|B000056KSU|     155726790| 2003-02-08|
|R100RJOC7TCXN6|   47602845|B005G492CC|     819279192| 2014-06-03|
|R10182KW3O6486|   52776942|B000031WEY|     719850793| 2004-04-23|
|R101B8W76YQILD|   12264663|B000BM7YYW|     364079538| 2006-03-10|
|R101K97ELZLSDK|   51870288|B00006NSDJ|     319548298| 2003-07-28|
|R101MMM4IXONBO|   43769541|B00C4XYEFS|     319009128| 2014-03-27|
|R101SUFE1N98DO|   51939266|B000002WR7|     467288526| 2002-10-27|
|R101Y7O5QCTMH4|   53039426|B000002GNQ|      31230172| 2001-03

In [9]:
review_id_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: string (nullable = true)



In [10]:
# Create a product dataframe for the database
product_df = music_df.select(["product_id", "product_title"])
product_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B000002J6H|      Stephen Stills|
|B00005BCY9|      Do You Love Me|
|B0000AB13N|        Project 1950|
|B000002SZA|My Christmas Favo...|
|B000056KSU|Os Sonhos Mas Lindos|
|B005G492CC|            The Help|
|B000031WEY|Bad Channels - So...|
|B000BM7YYW|       Walk the Line|
|B00006NSDJ|    Oh So Many Years|
|B00C4XYEFS|Bennett & Brubeck...|
|B000002WR7|If Every Day Was ...|
|B000002GNQ|          Wavelength|
|B000KEGH7Q|Bye-Bye Cigarette...|
|B0002VEU7Q|Diplomatic Immuni...|
|B00005M989|The Very Best of ...|
|B00F2HW20M|Genesis Revisited...|
|B00LH98PEG|      Ragged & Dirty|
|B000CQQHGI|The Definitive Co...|
|B00000AW34|         Full Circle|
|B001C0CJLU|Classic Album Col...|
+----------+--------------------+
only showing top 20 rows



In [11]:
product_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [12]:
# Create customer dataframe for the  database
customers_df = music_df.select(["customer_id"])
customers_df = customers_df.groupBy("customer_id").count()
customers_df.orderBy("customer_id").select(["customer_id", "count"])
customers_df = customers_df.withColumnRenamed("count", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   51893555|             1|
|   12945148|             5|
|   38897684|            21|
|   15261709|             3|
|   53036614|           125|
|   50509124|             2|
|   47996333|            23|
|   13560803|             3|
|   51680053|            23|
|   52037325|            14|
|   30682568|            58|
|   35871172|            88|
|   52599576|             3|
|   39162192|             6|
|   46859391|             1|
|   22421107|             6|
|   14219983|             4|
|   10944849|             1|
|   45936211|             1|
|    1592147|            23|
+-----------+--------------+
only showing top 20 rows



In [13]:
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [14]:
# Create vine dataframe for the  database
vine_df = music_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1005BINTGKCVE|          5|            0|          0|   N|
|R10067SN6FSII1|          4|            0|          0|   N|
|R100NGDZ6CAL8E|          1|            2|          2|   N|
|R100OLMAO039IL|          3|            2|          2|   N|
| R100OMDUXE5VZ|          4|            0|          0|   N|
|R100RJOC7TCXN6|          5|            2|          2|   N|
|R10182KW3O6486|          4|            5|          6|   N|
|R101B8W76YQILD|          4|            5|          5|   N|
|R101K97ELZLSDK|          4|            7|          9|   N|
|R101MMM4IXONBO|          5|            0|          0|   N|
|R101SUFE1N98DO|          5|            3|          6|   N|
|R101Y7O5QCTMH4|          4|            5|          7|   N|
|R1020J9O3H1F52|          5|            0|          0|   N|
|R1024GZRHXWW2F|          1|            

In [15]:
vine_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



Postgres Setup

In [16]:
# IMPORTANT: Replace each of these parameters with your own values for your AWS RDS instance
my_aws_endpoint = '' # replace with your own
my_aws_port_number = '' # Your value is likely the same, but please double check
my_aws_database_name = '' # please replace with your own
my_aws_username = '' # Your value is likely the same, but please double check
my_aws_password = ''# replace with your own


In [17]:
# Define the connection string
jdbc_url=f'jdbc:postgresql://{my_aws_endpoint}:{my_aws_port_number}/{my_aws_database_name}'

# Set up the configuration parameters
config = {"user": f'{my_aws_username}', 
          "password": f'{my_aws_password}', 
          "driver":"org.postgresql.Driver"}

# Choose to overwrite the existing data. Note that 'append' is probably a smarter choice
# in those situations where the PostgreSQL schema automatically generates the primary key. 
#  But 'overwrite' works best for this little demo. 
mode = 'overwrite' 


In [18]:
# Write the dataframe to the appropriate table in your PostgreSQL RDS

review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [19]:
# Write the dataframe to the appropriate table in your PostgreSQL RDS

product_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [20]:
# Write the dataframe to the appropriate table in your PostgreSQL RDS

customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [21]:
# Write the dataframe to the appropriate table in your PostgreSQL RDS

vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)